## criptografia assimétrica

### um exemplo em Python usando o algoritmo [RSA](https://pt.wikipedia.org/wiki/RSA)

em primeiro lugar, precisamos dois números primos, que chamamos aqui de $p$ e $q$.

neste exemplo, geramos uma lista dos números primos até um número $n$ que escolhemos. a lista é gerada usando um algoritmo desse [usuário](http://stackoverflow.com/users/350331) do stackoverflow (adaptado por esse [usuário](http://stackoverflow.com/users/5439078/jason)), mas poderíamos ter usado uma lista de primos gerada por qualquer meio.

no mundo real, o algoritmo RSA emprega números primos de ordens superiores à $10^{100}$, e por isso gera esses esses números de forma diferente do código abaixo: chuta-se um valor dessa ordem, e emprega-se um teste de primalidade com probabilidade praticamente zero de falso positivo; se o número passa no teste, usa-se o número, senão soma-se um ao número e faz-se o teste novamente.

In [1]:
import itertools
izip = itertools.zip_longest
chain = itertools.chain.from_iterable
compress = itertools.compress
def rwh_primes(n):
    """ Input n>=6, Returns a list of primes, 2 <= p < n """
    zero = bytearray([False])
    size = n//3 + (n % 6 == 2)
    sieve = bytearray([True]) * size
    sieve[0] = False
    for i in range(int(n**0.5)//3+1):
        if sieve[i]:
            k=3*i+1|1
            start = (k*k+4*k-2*k*(i&1))//3
            sieve[(k*k)//3::2*k]=zero*((size - (k*k)//3 - 1) // (2 * k) + 1)
            sieve[  start ::2*k]=zero*((size -   start  - 1) // (2 * k) + 1)
    ans = [2,3]
    poss = chain(izip(*[range(i, n, 6) for i in (1,5)]))
    ans.extend(compress(poss, sieve))
    return ans

gerando primos até 100.000.000 a título de exemplo.

In [2]:
prime_list = rwh_primes(100000000)
length = len(prime_list)
print("primos de 2 até 100.000:", prime_list[:20], "...", prime_list[length-20:])

primos de 2 até 100.000: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71] ... [99999587, 99999589, 99999611, 99999617, 99999623, 99999643, 99999677, 99999703, 99999721, 99999773, 99999787, 99999821, 99999827, 99999839, 99999847, 99999931, 99999941, 99999959, 99999971, 99999989]


aqui escolhemos dois primos da lista, aleatoriamente.

In [3]:
import random as rd
p = rd.choice(prime_list)
q = rd.choice(prime_list)
print("p =", p, ", q =", q)

p = 80572187 , q = 59697359


calculamos $n$, a multiplicação de $p$ e $q$, e o seu [tociente](https://pt.wikipedia.org/wiki/Fun%C3%A7%C3%A3o_totiente_de_Euler), definido por: 

$$\phi(n) = \phi(p \cdot q) = (p-1) \cdot (q-1)$$

o tociente de $n$ é a quantidade de números menores do que $n$ [co-primos](https://pt.wikipedia.org/wiki/N%C3%BAmeros_primos_entre_si) em relação à $n$. 

para um número primo $p$, é fácil ver que todos os números menores do que $p$ são co-primos em relação a $p$, e por isso:

$$ \phi(p) = (p-1), \forall p \in \mathbb{P}$$

como a função tociente $\phi$ é multiplicativa, vale a fórmula acima. (não ofereço prova)

In [4]:
n = p*q
phi = (p-1)*(q-1)
print("n =", n, ", phi =", phi)

n = 4809946772754133 , phi = 4809946632484588


aqui escolhemos $e$, um número primo que também seja coprimo em relação à $n$. como $e$ faz parte da chave pública, ele não precisa ser aleatório (e geralmente não o é), mas aqui escolhemos $e$ aleatoriamente.

In [5]:
from math import gcd

# it's fine if e is not random, because if a large number is picked, things may get slow during encryption
def find_coprime(p, q):
    n = p * q
    maxp = max(p, q)
    maxp_ix = prime_list.index(maxp)
    i = rd.choice(prime_list[:maxp_ix])
    not_found = True
    while not_found:
        if gcd(i, n) == 1:
            coprime = i
            not_found = False
        else:
            i = rd.choice(prime_list[:maxp_ix])
    return coprime

# the function below (in red) is the one usually employed:
'''def find_coprime(n):
    # not using this function because I'm not choosing primes garanteed to be greater than 65537
    i = 65537
    not_found = True
    while not_found:
        if gcd(i, n) == 1:
            coprime = i
            not_found = False
        else:
            prime_ind = prime_list.index(i)
            i = prime_list(prime_ind + 1)
    return coprime'''

e = find_coprime(p, q)
print("e =", e)

e = 38483509


agora, precisamos de $d$ para a chave privada. $d$ é o inverso multiplicativo de $e$ em relação à $\phi (n)$, i.e., $d$ é o número tal que:

$$e\cdot d = 1 \bmod \phi(n)$$

a fórmula acima diz que $e \cdot d$ dividido por $\phi (n)$ tem resto 1.

tendo $e$, para achar $d$ usamos o [algoritmo de euclides extendido](https://pt.wikipedia.org/wiki/Algoritmo_de_Euclides_estendido):

In [6]:
import sys
sys.setrecursionlimit(1000000)  # long type,32bit OS 4B,64bit OS 8B(1bit for sign)
# code in this cell from: https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm

def egcd(a, b):
# return (g, x, y) in a*x + b*y = g = gcd(x, y)
    #print("egcd called using a = " + str(a) + " and b = " + str(b))
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y = egcd(b % a, a)
        #print("intermediate result, g = " + str(g) + " and x = " + str(y - (b // a) * x) + " and y = " + str(x))
        return (g, y - (b // a) * x, x)

def mulinv(b, n):
# x = mulinv(b) mod n, (x * b) % n == 1
    g, x, _ = egcd(b, n)
    if g == 1:
        return x % n

In [7]:
d = mulinv(e, phi)
print("d =", d)

d = 2589194910363685


conferindo o resultado:

In [8]:
e*d % phi

1

agora podemos começar a aplicar o algoritmo RSA propriamente dito. montamos as chaves pública e privada da seguinte forma:

$\mathit{ChavePub} = (e, n)$ e $\mathit{ChavePriv} = (d, n)$.

In [9]:
pub_key = (e, n)
print("chave pública =", pub_key)

chave pública = (38483509, 4809946772754133)


In [10]:
pri_key = (d, n)
print("chave privada =", pri_key)

chave privada = (2589194910363685, 4809946772754133)


agora escrevemos as funções de encriptação e decriptação:

$$ E(m, \mathit{ChavePub}) \equiv E(m, e, n) = m^e \bmod n = c $$

$$ D(c, \mathit{ChavePriv}) \equiv D(c, d, n) = c^d \bmod n = m $$

em que $m$ é a mensagem pura, $c$ é a mensagem cifrada, e $n$ é a multiplicação dos números primos $p$ e $q$.

In [11]:
def encrypt(m, pub_key):
    e, n = pub_key
    #print(e, n)
    '''ciphertext = cipher(plaintext)
    m = int(ciphertext)'''
    c = pow(m, e, n) # same as (m**e) % n
    return c

In [12]:
def decrypt(c, pri_key):
    d, n = pri_key
    #print(d, n)
    m = pow(c, d, n) # same as (c**d) % n
    return m

# the function below (in red) is a more efficient way of decrypting a ciphertext, using the chinese remainder theorem.
'''def decrypt2(c, pri_key):
    # uses Chinese Remainder Theorem to speed up calculations, thanks to Joao H de A Franco at https://jhafranco.com
    d, n = pri_key
    #print(d, n)
    m1 = pow(c, d % (p - 1), p)
    m2 = pow(c, d % (q - 1), q)
    h = (mulinv(q, p) * (m1 - m2)) % p
    m = m2 + h * q
    #plaintext = decipher(m)
    return m'''

'def decrypt2(c, pri_key):\n    # uses Chinese Remainder Theorem to speed up calculations, thanks to Joao H de A Franco at https://jhafranco.com\n    d, n = pri_key\n    #print(d, n)\n    m1 = pow(c, d % (p - 1), p)\n    m2 = pow(c, d % (q - 1), q)\n    h = (mulinv(q, p) * (m1 - m2)) % p\n    m = m2 + h * q\n    #plaintext = decipher(m)\n    return m'

imaginemos agora que Babbage queira enviar por correio para Ada a senha para o uso de seu computador analítico. como seu computador tem arquivos confidenciais, Babbage não quer colocar a senha pura no papel; ele quer encriptá-la para que só Ada possa lê-la. Babbage encripta a senha do seu computador ($m$) usando a chave-pública de Ada, $e$ e $n$. a chave-pública é algo que Ada distribui para os seus amigos. (assim como ela deu seu endereço para Babbage, ele também deu sua chave-pública; é o equivalente atual de dizer seu endereço de email para um amigo ou colega)

Babbage faz: 

$$ E(m, e, n) = m^e \bmod n = c $$

i.e., Babbage eleva $m$ à $e$-esima potência, divide o valor encontrado por n, e encontra o resto da divisão. esse resto é a mensagem cifrada $c$. Babbage escreve $c$ na sua carta para Ada.

com $m = 26121791$, Babbage faz:

In [13]:
c = encrypt(26121791, pub_key)
print("c =", c)

c = 575073478552795


[algumas horas depois](http://nyti.ms/1ABDhKp), Ada recebe a carta de Babbage. Ada recolhe a mensagem cifrada $c$. para decifrá-la, Ada usa sua chave-privada $d$ e $n$. só Ada conhece o valor $d$. Ada faz:

$$ D(c, d, n) = c^d \bmod n = m $$

i.e., Ada eleva a mensagem cifrada $c$ à $d$-ésima potência, divide o valor encontrado por n, e encontra o resto da divisão. esse resto é a mensagem pura original $m$, a senha para o computador analítico de Babbage. isso só ocorre por conta das propriedades matemáticas dos números e das operações escolhidas.

In [14]:
m = decrypt(c, pri_key)
m

26121791

Ada digita a senha, e voilà, o computador analítico está a sua disposição.

___

por um descuido, Ada não queima a carta enviada por Babbage. ela vai para o lixo, e por um acaso, vai parar na mão de um inimigo de Babbage, Epshank. Epshank lê a carta e descobre que $c$ é a forma cifrada da senha do computador analítico de Babbage. Epshank quer muito decifrar a senha, e ele sabe a chave-pública de Ada que foi usada para encriptar a mensagem (afinal, a chave-pública é pública). para tentar adivinhar a chave-privada de Ada, Epshank precisar encontrar $d$, já que $n$ faz parte da chave-pública que ele conhece. Ora, $d$ é o inverso multiplicativo de $e$, que Epshank conhece. $d$ obedece à relação $e \cdot d = 1 \bmod \phi (n)$, o que Epshank também sabe. Epshank não sabe quanto é $\phi (n)$, mas sabe que $\phi (n) = (p-1) \cdot (q-1)$, em que $p$ e $q$ são os números primos que compõem $n$. Epshank puxa de sua memória como se fatora um número:

<img src="fatoracao.png" alt="fatoração de um número" width="250" >

"ora, parece fácil.", pensou Epshank. mas quando Epshank tentou fatorar $n$, ele desistiu.

In [15]:
print("n =", n)

n = 4809946772754133


$n$ é um número MUITO grande. no RSA implantado no mundo real, $n$ seria um número superior a $2^{2047}.

a segurança do algoritmo RSA reside nesse simples fato: é fácil calcular $n$ ou $\phi (n)$ dados dois números primos $p$ e $q$, mas achar $p$ e $q$ dado $n$ é muito mais difícil.

um exemplo que pode ser calculado com papel e lápis é os seguinte. calcule $n$ dado $p = 419$ e $q = 521$. é fácil achar o valor de $n$ nesse caso, sem usar calculadora alguma (vá lá, tente.)

agora, se eu lhe der o número $183731$ e lhe pedir para fatorá-lo, você vai levar mais tempo. se eu lhe der um número com 617 dígitos então, até mesmo um computador vai levar mais de décadas para resolver o problema usando força bruta.

Fontes:

http://doctrina.org/How-RSA-Works-With-Examples.html
    
http://logos.cs.uic.edu/340%20notes/rsa.html
    
https://www.cs.utexas.edu/~mitra/honors/soln.html